# Marketing Assistant

Copyright 2025, Denis Rothman

**Goal:** This notebook serves as the practical runtime for the **Strategic Marketing Engine**, as architected in Chapter 9. It demonstrates how the generic, multi-domain "Glass Box" Context Engine can be seamlessly repurposed to solve a variety of real-world marketing challenges without any changes to its core code.

This notebook will:
* Connect to the Pinecone knowledge base populated by the `Data_Ingestion_Marketing.ipynb` script.
* Use the generic Control Deck templates to interact with the engine.
* Execute the marketing use cases




# I. Inititalization

## GitHub

In [1]:
print("Downloading files from public repository...")

# The -f flag tells curl to fail on an error (like 404)
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/utils.py --output utils.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/helpers.py --output helpers.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/agents.py --output agents.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/registry.py --output registry.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/engine.py --output engine.py
# (You might want to add a check here to see if the files actually exist now)
print("✅ File download attempt finished!")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1823  100  1823    0     0  10502      0 --:--:-- --:--:-- --:--:-- 10537
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6479  100  6479    0     0  33617      0 --:--:-- --:--:-- --:--:-- 33744
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9440  100  9440    0     0  45585      0 --:--:-- --:--:-- --:--:-- 45603
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3611  100  3611    0     0  19760      0 --:--:-- --:--:-- --:--:-- 19840
  % Total    % Received % Xferd  Average Speed   Tim

## Installation and client setup

In [2]:
#Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

🚀 Installing required packages...
✅ All packages installed successfully.

🔑 Initializing API clients...
   - OpenAI client initialized.
   - Pinecone client initialized.
✅ Clients initialized successfully.


## Context Engine library Import

In [3]:
# 1. Import the hardened helper functions (LLM, Embeddings, Pinecone)
import helpers

# 2. Import the specialist agent functions (Librarian, Researcher, Writer)
import agents

# 3. Import the AGENT_TOOLKIT object that knows about all the agents
from registry import AGENT_TOOLKIT

# 4. Import the main context_engine function that orchestrates the entire process
from engine import context_engine

## Engine Room

In [4]:
# === ENGINE ROOM: The Main Execution Function ===
# This function contains all the logic to run the engine.
# We define it here so our final cell can be very simple.

import logging
import pprint
from IPython.display import display, Markdown

# In Legal_Compliance_Assistant.ipynb (The "Engine Room" cell)

def execute_and_display(goal, config, client, pc, moderation_active):
    """
    Runs the context engine, now with an optional, two-stage moderation check.
    """
    # --- PRE-FLIGHT MODERATION CHECK (on user input) ---
    if moderation_active:
        print("--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---")
        moderation_report = helpers.helper_moderate_content(text_to_moderate=goal, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Goal failed pre-flight moderation. Execution halted.")
            return # Halt execution before calling the engine

    logging.info(f"******** Starting Engine for Goal: '{goal}' **********\\n")

    # 1. Run the Context Engine using the provided configuration
    result, trace = context_engine(
        goal,
        client=client,
        pc=pc,
        **config
    )

    # --- POST-FLIGHT MODERATION CHECK (on AI output) ---
    if result and moderation_active:
        print("\n--- [Safety Guardrail] Performing Post-Flight Moderation Check on Output ---")
        moderation_report = helpers.helper_moderate_content(text_to_moderate=result, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Generated output failed post-flight moderation and will be redacted.")
            result = "[Content flagged as potentially harmful by moderation policy and has been redacted.]"

    # 2. Display the Final Result
    print("\n--- FINAL OUTPUT ---")
    if result:
        display(Markdown(result))
    else:
        print(f"The engine failed to produce a result. Status: {trace.status}")

    # 3. Display the Technical Trace
    print("\\n\\n--- TECHNICAL TRACE (for the tech reader) ---")
    if trace:
        print(f"Trace Status: {trace.status}")
        print(f"Total Duration: {trace.duration:.2f} seconds")
        print("Execution Steps:")
        # --- THIS LINE IS TO CREATE THE 'pp' OBJECT ---
        pp = pprint.PrettyPrinter(indent=2)

        pp.pprint(trace.steps)

## Control Deck configuration

In [5]:
# 1. Define all configuration variables for this run in a dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

#III.CONTROL DECKS

=== CONTROL DECK: Define Goal and Run Engine ===
This is the main interactive cell.
1. Change the 'goal' variable to your desired task.
2. Run this cell.


In [6]:
#@title CONTROL DECK: Moderation
# 1. Define a simple, safe goal to test the moderation workflow.
goal = "Summarize the key points of the QuantumDrive"

# 2. Define the standard configuration.
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function with moderation explicitly activated.
execute_and_display(goal, config, client, pc, moderation_active=True)

--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---
Moderation Report:
{'categories': {'harassment': False,
                'harassment/threatening': False,
                'harassment_threatening': False,
                'hate': False,
                'hate/threatening': False,
                'hate_threatening': False,
                'illicit': False,
                'illicit/violent': False,
                'illicit_violent': False,
                'self-harm': False,
                'self-harm/instructions': False,
                'self-harm/intent': False,
                'self_harm': False,
                'self_harm_instructions': False,
                'self_harm_intent': False,
                'sexual': False,
                'sexual/minors': False,
                'sexual_minors': False,
                'violence': False,
                'violence/graphic': False,
                'violence_graphic': False},
 'flagged': False,
 'scores': {'harassment': 4

ERROR:root:An error occurred during content moderation: Error code: 400 - {'error': {'message': "[{'type': 'string_type', 'loc': ('body', 'input', 'str'), 'msg': 'Input should be a valid string', 'input': {'summary': '- The provided material contains no reference to any product, technology, or specification named “QuantumDrive.”  \\n- It therefore does not define what QuantumDrive is, what problem it solves, or how it works (architecture/mechanisms).  \\n- No benefits, limitations, or usage considerations for QuantumDrive can be derived from the text.  \\n- Any description of QuantumDrive based on this material would be speculative and not supported by the sources.'}}, {'type': 'list_type', 'loc': ('body', 'input', 'list[str]'), 'msg': 'Input should be a valid list', 'input': {'summary': '- The provided material contains no reference to any product, technology, or specification named “QuantumDrive.”  \\n- It therefore does not define what QuantumDrive is, what problem it solves, or how


--- [Safety Guardrail] Performing Post-Flight Moderation Check on Output ---
Moderation Report:
{'categories': {'error': "Error code: 400 - {'error': {'message': "
                         '"[{\'type\': \'string_type\', \'loc\': (\'body\', '
                         "'input', 'str'), 'msg': 'Input should be a valid "
                         "string', 'input': {'summary': '- The provided "
                         'material contains no reference to any product, '
                         'technology, or specification named “QuantumDrive.”  '
                         '\\\\n- It therefore does not define what '
                         'QuantumDrive is, what problem it solves, or how it '
                         'works (architecture/mechanisms).  \\\\n- No '
                         'benefits, limitations, or usage considerations for '
                         'QuantumDrive can be derived from the text.  \\\\n- '
                         'Any description of QuantumDrive based on this '

[Content flagged as potentially harmful by moderation policy and has been redacted.]

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 8.84 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': 'Based on the provided sources, there '
                                       'is no mention or description of any '
                                       'product, technology, or specification '
                                       'called “QuantumDrive.” The texts '
                                       'discuss:\n'
                                       '\n'
                                       '- Historical human spaceflight (Apollo '
                                       'program, Apollo 11 landing).\n'
                                       '- A specific Jupiter mission (NASA’s '
                                       'Juno probe and its solar-powered '
                                       'design).\n'
                                       '- Mars rovers (Sojourner, Spirit, '
            

In [7]:
#@title Product Marketing Copy Generation(Use Case 1)
goal = "Analyze the ChronoTech press release and summarize their core product messaging and value proposition. Please cite your sources."


# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


## Core Product Messaging

The provided source material contains no information about ChronoTech—its products, services, markets, customers, or positioning. All available text concerns NASA space exploration programs and does not mention ChronoTech at all. As a result, it is not possible to produce an evidence-based product description or authentic messaging pillars for ChronoTech without speculating beyond the source.

- Main message pillars: Not derivable from the source material  
  - Product capabilities: Not described  
  - Target users or industries: Not described  
  - Key benefits (e.g., speed, accuracy, automation, security): Not described  
  - Differentiators vs. competitors: Not described  

Any attempt to define ChronoTech’s core product messaging from the given content would be hypothetical and not grounded in the cited material.

## Value Proposition

Because the source material does not contain any reference to ChronoTech, its offerings, or its customers, a valid value proposition cannot be constructed. There is no factual basis in the provided text to describe how ChronoTech creates value, what problems it solves, or for whom.

- Benefit-to-segment mapping: Not available  
  - Customer segments: Not identified in the source  
  - Use cases: Not identified  
  - Outcomes (cost savings, time savings, risk reduction, etc.): Not identified  

Any specific value statements for ChronoTech would be conjecture and would not accurately reflect the source.

## Source Citations

- Composite source: “Space exploration overview, Apollo program, Juno mission, and Mars rover description (single composite source as provided).”  
  - Publisher: Not specified in the provided material  
  - Date: Not specified in the provided material  
  - URL: Not specified in the provided material  

Note: The composite source explicitly states that there is no information about ChronoTech or any ChronoTech press releases in the provided content.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 19.31 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': 'There is no information about '
                                       'ChronoTech or any ChronoTech press '
                                       'releases in the provided sources, so I '
                                       'cannot extract or synthesize a core '
                                       'product description, messaging, or '
                                       'value proposition for ChronoTech from '
                                       'them.\n'
                                       '\n'
                                       'All of the supplied text is about '
                                       'historical and current space '
                                       'exploration activities by NASA and '
                                       'related missions (Apollo, Juno, 

In [8]:
#@title Product Marketing Copy Generation(Use Case 2)


# 1. Define the Goal: A research query that requires a verifiable, cited answer.
#    - DOMAIN: Any knowledge-intensive field (e.g., legal, medical, financial).
#    - KEY CAPABILITY: Tests the high-fidelity `Researcher` agent and its ability
#      to retrieve text with `source` metadata and generate citations.
# goal = "[INSERT YOUR HIGH-FIDELITY RESEARCH GOAL HERE]"

## 1. Define the Goal: A research query that asks for a creative output.
goal = "Using the official product spec sheet, write a short marketing description for the new QuantumDrive Q-1. The description should be confident, aspirational, and focus on the benefits for creative professionals. Please cite your sources."



# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


So, quick rundown:

The info you gave me is all about space stuff—Sputnik, Apollo 11, the Juno mission, Mars rovers like Perseverance, and the Ingenuity helicopter. None of it talks about any kind of storage device, and definitely not something called the “QuantumDrive Q-1.”

Because of that, I can’t pull real specs or details for QuantumDrive Q-1 from those materials. If you’ve got actual docs, marketing text, or tech notes about that product, I could use those to put together a proper spec sheet or description. Right now, though, the sources are 100% space history, 0% storage hardware.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 13.88 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': 'There is no information in the '
                                       'provided sources about a product '
                                       'called the "QuantumDrive Q-1" storage '
                                       'device, so I cannot give an official '
                                       'product spec sheet or technical '
                                       'details for it based on these texts.\n'
                                       '\n'
                                       'All of the supplied material concerns '
                                       'space exploration history and missions '
                                       '(Sputnik, Apollo 11, Juno, Mars rovers '
                                       'such as Perseverance and the Ingenuity '
                          

In [9]:
#@title Writing a brand pitch recommendation(Use case 3)

# 1. Define the Goal: A creative or factual task that is deliberately
#    outside the scope of the documents in the knowledge base.
#    - DOMAIN: Universal test applicable to any curated knowledge base.
#    - KEY CAPABILITY: Tests the `Researcher` agent's ability to report a
#      negative finding and the `Writer` agent's ability to handle it gracefully,
#      preventing hallucination.
# goal = "[INSERT YOUR OUT-OF-SCOPE GOAL HERE]"


# =CONTROL DECK 3: A persuasive pitch ===
goal = "Write a persuasive pitch on our brand tone and voice guide"


# 2. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


A brand tone and voice guide is basically your company’s “this is how we talk” rulebook. It takes all the high-level brand stuff—your purpose, values, positioning, audience—and turns it into real, usable guidelines for language: how you sound, what you say, and how you adapt in different situations. Voice = who you are. Tone = how you show up in different contexts (email vs. social, sales vs. crisis, etc.).

Why it matters: it makes your brand feel like one consistent “person” everywhere—website, ads, product, support, leadership. That consistency builds trust, makes you stand out, and keeps everyone on the same page so you’re not constantly debating wording. It’s how big-picture strategy becomes everyday behavior.

If you don’t have (or don’t follow) one, everything starts to feel messy. Your ads, product, and support sound like different companies. Tone can swing from fun to stiff, or caring to robotic, which hurts credibility and makes it harder to justify premium pricing. Internally, you get more rewrites, misaligned campaigns, slow onboarding, and awkward handling of sensitive issues like outages or complaints.

When you *do* have a solid guide, the benefits show up everywhere:

- Organization-wide: Content gets created faster with fewer review loops. The brand feels sharper and more valuable over time, which helps with growth and pricing power.
- Marketing: Campaigns feel cohesive across channels and partners. Agencies and freelancers know exactly how to write, so you get better work, faster, with stronger recall and engagement.
- Sales: Emails and proposals feel on-brand but still human. The transition from marketing to sales is smoother, boosting trust and conversion, and keeping promises aligned with the brand story.
- Product & UX: In-app copy is clear, calm, and consistent with marketing. The product feels more polished and intuitive just because the words are well thought out.
- Customer Support: Agents respond in a consistent, empathetic way, which helps with complaints, escalations, CSAT, and loyalty. Even in tough moments, the tone supports the brand instead of hurting it.
- Leadership: Leaders have a clear way to express strategy and values in all comms. It sends a strong cultural signal internally (“this is how we show up”) and protects the brand as you scale, acquire, or change.

In short: a tone and voice guide is how you make your brand sound like one confident, trustworthy personality everywhere, without reinventing the wheel every time someone writes an email, ad, or help article.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 60.76 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': 'An effective brand tone and voice '
                                       'guide defines how a brand “sounds” in '
                                       'every interaction and provides '
                                       'concrete rules to keep that sound '
                                       'consistent across teams, channels, and '
                                       'time. It matters because it directly '
                                       'shapes stakeholder alignment, message '
                                       'consistency, customer trust, and '
                                       'long‑term brand equity.\n'
                                       '\n'
                                       'Because the provided sources only '
                                       'cover s